# Import Dependencies & Dataset

In [1]:
import pandas as pd
import numpy as np
import re
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from scipy.sparse import csr_matrix

In [2]:
df = pd.read_excel('dataset.xlsx')
df

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...
522059,581587,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
522060,581587,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
522061,581587,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
522062,581587,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


There are 7 features & 522064 orders of data included in this dataset.

the features are :

- BillNo
- Itemname
- Quantity
- Date
- CustomerID
- Country

and for this project, we will only use 'BillNo' and 'Itemname' features to seek a purchasing pattern of a customer.

# Dataset Exploration

In [3]:
arr_itemName = df['Itemname'].unique()
for item in arr_itemName : print(item)

WHITE HANGING HEART T-LIGHT HOLDER
WHITE METAL LANTERN
CREAM CUPID HEARTS COAT HANGER
KNITTED UNION FLAG HOT WATER BOTTLE
RED WOOLLY HOTTIE WHITE HEART.
SET 7 BABUSHKA NESTING BOXES
GLASS STAR FROSTED T-LIGHT HOLDER
HAND WARMER UNION JACK
HAND WARMER RED POLKA DOT
ASSORTED COLOUR BIRD ORNAMENT
POPPY'S PLAYHOUSE BEDROOM
POPPY'S PLAYHOUSE KITCHEN
FELTCRAFT PRINCESS CHARLOTTE DOLL
IVORY KNITTED MUG COSY
BOX OF 6 ASSORTED COLOUR TEASPOONS
BOX OF VINTAGE JIGSAW BLOCKS
BOX OF VINTAGE ALPHABET BLOCKS
HOME BUILDING BLOCK WORD
LOVE BUILDING BLOCK WORD
RECIPE BOX WITH METAL HEART
DOORMAT NEW ENGLAND
JAM MAKING SET WITH JARS
RED COAT RACK PARIS FASHION
YELLOW COAT RACK PARIS FASHION
BLUE COAT RACK PARIS FASHION
BATH BUILDING BLOCK WORD
ALARM CLOCK BAKELIKE PINK
ALARM CLOCK BAKELIKE RED
ALARM CLOCK BAKELIKE GREEN
PANDA AND BUNNIES STICKER SHEET
STARS GIFT TAPE
INFLATABLE POLITICAL GLOBE
VINTAGE HEADS AND TAILS CARD GAME
SET/2 RED RETROSPOT TEA TOWELS
ROUND SNACK BOXES SET OF4 WOODLAND
SPACEBOY LUN

In [4]:
len(arr_itemName)

4186

In [5]:
df['Itemname'].isna().sum()

1455

as we can see here, there are 4186 unique items in the list whereas there are still some of them is not a valid item, such as :
- "?missing"
- "??"
- "????damages????"
- "lost??"

and some of them are a string that is not even a valid item, examples :

- "wet"
- "lost"

also there are 1455 *not available* Itemname in this dataset.

In [6]:
len(df['BillNo'].unique())

21663

there are 21,663 transactions in this dataset

*some of them will be removed, due to invalid name of items*

# Data Preprocessing

### Dropping features

In [7]:
df = df.drop(columns=['Quantity','Date','Price','CustomerID','Country'])

as we mentioned earlier, we will only use the 'BillNo' and 'Itemname' features for this project to seek a purchasing pattern of a customer

### Itemname -- *dropping not available values*

In [8]:
df = df.dropna()
df

,BillNo,Itemname
0,536365,WHITE HANGING HEART T-LIGHT HOLDER
1,536365,WHITE METAL LANTERN
2,536365,CREAM CUPID HEARTS COAT HANGER
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,RED WOOLLY HOTTIE WHITE HEART.
...,...,...
522059,581587,PACK OF 20 SPACEBOY NAPKINS
522060,581587,CHILDREN'S APRON DOLLY GIRL
522061,581587,CHILDRENS CUTLERY DOLLY GIRL
522062,581587,CHILDRENS CUTLERY CIRCUS PARADE


since there are some *na* values in the Itemname feature, we should remove it

### Itemname --*removing rows that contains special characters*

In [9]:
def remove_rows_with_special_chars(df, column_name):
    regex_pattern = r'[^a-zA-Z0-9\s]'  
    mask = df[column_name].str.contains(regex_pattern)  
    cleaned_df = df[~mask]  
    return cleaned_df

In [10]:
df = remove_rows_with_special_chars(df,'Itemname')
df

,BillNo,Itemname
1,536365,WHITE METAL LANTERN
2,536365,CREAM CUPID HEARTS COAT HANGER
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE
5,536365,SET 7 BABUSHKA NESTING BOXES
7,536366,HAND WARMER UNION JACK
...,...,...
522058,581587,CHILDRENS CUTLERY SPACEBOY
522059,581587,PACK OF 20 SPACEBOY NAPKINS
522061,581587,CHILDRENS CUTLERY DOLLY GIRL
522062,581587,CHILDRENS CUTLERY CIRCUS PARADE


as we mentioned earlier that there are some item names that includes some special chars, which are not representate the item itself, so we remove it.

### saving each items into arr_item before we group the items

In [11]:
arr_item = df['Itemname'].unique()

### Grouping items per-transaction

In [12]:
def group_items_by_bill(df):
    grouped_items = df.groupby('BillNo')['Itemname'].apply(list).reset_index()
    return grouped_items

df_grouped = group_items_by_bill(df)
print(df_grouped)

        BillNo                                           Itemname
0       536365  [WHITE METAL LANTERN, CREAM CUPID HEARTS COAT ...
1       536366  [HAND WARMER UNION JACK, HAND WARMER RED POLKA...
2       536367  [ASSORTED COLOUR BIRD ORNAMENT, FELTCRAFT PRIN...
3       536368  [JAM MAKING SET WITH JARS, RED COAT RACK PARIS...
4       536369                         [BATH BUILDING BLOCK WORD]
...        ...                                                ...
19834   581586  [LARGE CAKE STAND  HANGING STRAWBERY, SET OF 3...
19835   581587  [CIRCUS PARADE LUNCH BOX, PLASTERS IN TIN CIRC...
19836  A563185                                  [Adjust bad debt]
19837  A563186                                  [Adjust bad debt]
19838  A563187                                  [Adjust bad debt]

[19839 rows x 2 columns]


# Data Modelling

In [13]:
df_grouped = df_grouped.drop(columns=['BillNo'])
df_grouped

,Itemname
0,"[WHITE METAL LANTERN, CREAM CUPID HEARTS COAT ..."
1,"[HAND WARMER UNION JACK, HAND WARMER RED POLKA..."
2,"[ASSORTED COLOUR BIRD ORNAMENT, FELTCRAFT PRIN..."
3,"[JAM MAKING SET WITH JARS, RED COAT RACK PARIS..."
4,[BATH BUILDING BLOCK WORD]
...,...
19834,"[LARGE CAKE STAND HANGING STRAWBERY, SET OF 3..."
19835,"[CIRCUS PARADE LUNCH BOX, PLASTERS IN TIN CIRC..."
19836,[Adjust bad debt]
19837,[Adjust bad debt]


In [14]:
item_to_index = {item: i for i, item in enumerate(arr_item)}

In [15]:
sparse_data = []
for transaction in df_grouped['Itemname']:
    row = [0] * len(arr_item)
    for item in transaction:
        row[item_to_index[item]] = 1
    sparse_data.append(row)
sparse_matrix = csr_matrix(sparse_data)
print(sparse_matrix)

  (0, 0)	1
  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (1, 4)	1
  (1, 5)	1
  (2, 6)	1
  (2, 7)	1
  (2, 8)	1
  (2, 9)	1
  (2, 10)	1
  (2, 11)	1
  (2, 12)	1
  (2, 13)	1
  (2, 14)	1
  (2, 15)	1
  (3, 16)	1
  (3, 17)	1
  (3, 18)	1
  (3, 19)	1
  (4, 20)	1
  (5, 21)	1
  (5, 22)	1
  (5, 23)	1
  (5, 24)	1
  :	:
  (19833, 2858)	1
  (19833, 3181)	1
  (19833, 3200)	1
  (19833, 3219)	1
  (19834, 270)	1
  (19834, 873)	1
  (19834, 1365)	1
  (19834, 2948)	1
  (19835, 21)	1
  (19835, 22)	1
  (19835, 23)	1
  (19835, 29)	1
  (19835, 31)	1
  (19835, 175)	1
  (19835, 366)	1
  (19835, 606)	1
  (19835, 607)	1
  (19835, 1723)	1
  (19835, 2345)	1
  (19835, 2723)	1
  (19835, 2724)	1
  (19835, 2725)	1
  (19836, 3192)	1
  (19837, 3192)	1
  (19838, 3192)	1


In [21]:
dense_matrix = sparse_matrix.toarray()
dense_df = pd.DataFrame(dense_matrix, columns=arr_item)
frequent_itemsets = apriori(dense_df, min_support=0.03, use_colnames=True)
frequent_itemsets.sort_values(by='support', ascending=False, inplace=True)
print(frequent_itemsets)

/Users/apple/anaconda3/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


      support                              itemsets
35   0.104038             (JUMBO BAG RED RETROSPOT)
66   0.095973            (REGENCY CAKESTAND 3 TIER)
87   0.083472                       (PARTY BUNTING)
18   0.077675             (LUNCH BAG RED RETROSPOT)
0    0.072131       (ASSORTED COLOUR BIRD ORNAMENT)
..        ...                                   ...
106  0.030495  (SET OF 60 PANTRY DESIGN CAKE CASES)
80   0.030445                (HOT BATHS METAL SIGN)
86   0.030445          (TRADITIONAL KNITTING NANCY)
28   0.030294           (PAPER CHAIN KIT RETROSPOT)
62   0.030243     (60 CAKE CASES VINTAGE CHRISTMAS)

[118 rows x 2 columns]


In [20]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

rules.sort_values(by='confidence', ascending=False, inplace=True)
print(rules.head())

                         antecedents                        consequents  \
2   (PINK REGENCY TEACUP AND SAUCER)  (GREEN REGENCY TEACUP AND SAUCER)   
1  (GREEN REGENCY TEACUP AND SAUCER)  (ROSES REGENCY TEACUP AND SAUCER)   
0  (ROSES REGENCY TEACUP AND SAUCER)  (GREEN REGENCY TEACUP AND SAUCER)   

   antecedent support  consequent support   support  confidence       lift  \
2            0.037099            0.049146  0.030495    0.822011  16.726024   
1            0.049146            0.051061  0.036847    0.749744  14.683280   
0            0.051061            0.049146  0.036847    0.721619  14.683280   

   leverage  conviction  zhangs_metric  
2  0.028672    5.342205       0.976437  
1  0.034337    3.791867       0.980061  
0  0.034337    3.415658       0.982039  


# Analysis

**RULE 1**

If customers purchase a "PINK REGENCY TEACUP AND SAUCER", there is a high likelihood (confidence of 82.20%) that they will also purchase a "GREEN REGENCY TEACUP AND SAUCER". The lift of 16.73 indicates a strong positive association between these items, meaning that the presence of one item significantly increases the likelihood of the other being purchased.

**RULE 2**

If customers purchase a "GREEN REGENCY TEACUP AND SAUCER", there is a substantial probability (confidence of 74.97%) that they will also purchase a "ROSES REGENCY TEACUP AND SAUCER". The lift of 14.68 indicates a strong positive association between these items, similar to Rule 1.

**RULE 3**

Rule 3: If customers purchase a "ROSES REGENCY TEACUP AND SAUCER", there is also a high likelihood (confidence of 72.16%) that they will purchase a "GREEN REGENCY TEACUP AND SAUCER". The lift of 14.68 again suggests a significant positive association between these items.

# Conclusion

These association rules provide valuable insights into customer purchasing behavior for tea cups and saucers. Specifically, there appears to be a strong tendency for customers who purchase one type of tea cup and saucer to also purchase another type, indicating potential cross-selling opportunities. Retailers could leverage these findings to optimize product placement, promotions, and marketing strategies to increase sales and enhance the overall shopping experience for customers.